In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive
df = pd.read_csv('/content/drive/MyDrive/Laptop_price.csv')

In [21]:
# Разделение на целевой столбец и признаки
X = df.drop('Price', axis=1)
y = df['Price']

In [22]:
# Деление на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Определение категориальных и количественных признаков
categoricalFeatures = X.select_dtypes(include=['object']).columns.tolist()
numericalFeatures = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [24]:
numericalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])

In [26]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(random_state=42))
])

In [27]:
# Задание сетки гиперпараметров для GridSearchCV
param = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 5, 7],
    'model__gamma': [0, 0.1, 0.2],
    'model__subsample': [0.8, 1.0]  
}

In [28]:
gridSearch = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, verbose=2)

In [ ]:
gridSearch.fit(X_train, y_train)

In [30]:
model = gridSearch.best_estimator_
bestParams = gridSearch.best_params_

In [ ]:
print("Лучшие параметры: ", bestParams)